In [1]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [10]:
print("Hello Payal")

Hello Payal


In [22]:
# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [23]:

words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

41 documents
15 classes ['address', 'courses', 'created', 'friend', 'goodbye', 'goodmorning', 'goodnight', 'greeting', 'hours', 'online', 'opentoday', 'payments', 'rental', 'thanks', 'today']
69 unique stemmed words ["'d", "'s", 'a', 'acceiv', 'address', 'al', 'anyon', 'ar', 'best', 'bye', 'can', 'card', 'cash', 'cours', 'cre', 'credit', 'day', 'develop', 'do', 'doe', 'friend', 'gm', 'gn', 'good', 'goodby', 'gud', 'hav', 'hello', 'help', 'hi', 'hour', 'how', 'i', 'is', 'kind', 'know', 'lat', 'lik', 'mastercard', 'may', 'mop', 'morn', 'nee', 'night', 'of', 'off', 'on', 'onlin', 'op', 'provid', 'rent', 'see', 'tak', 'thank', 'that', 'ther', 'thi', 'to', 'today', 'train', 'we', 'what', 'when', 'wher', 'which', 'who', 'work', 'yo', 'you']


In [24]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [25]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 5999  | total loss: 0.29291 | time: 0.058s
| Adam | epoch: 1000 | loss: 0.29291 - acc: 0.9849 -- iter: 40/41
Training Step: 6000  | total loss: 0.26470 | time: 0.070s
| Adam | epoch: 1000 | loss: 0.26470 - acc: 0.9864 -- iter: 41/41
--
INFO:tensorflow:C:\Users\omc\Desktop\Payal_v1.0\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [26]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [27]:
p = bow("is your shop open today?", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0]
['address', 'courses', 'created', 'friend', 'goodbye', 'goodmorning', 'goodnight', 'greeting', 'hours', 'online', 'opentoday', 'payments', 'rental', 'thanks', 'today']


In [28]:
print(model.predict([p]))

[[  1.39111811e-02   5.37071899e-02   3.10503111e-07   1.00934121e-05
    1.09405009e-12   2.50055723e-17   2.11466209e-11   1.04891918e-13
    5.83979698e-10   9.25652862e-01   1.86997335e-04   2.74654681e-08
    2.12357919e-14   3.83910119e-05   6.49286807e-03]]


In [29]:

# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )